In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import os
import pandas as pd
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow import keras
from keras import metrics


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import gdown
import os, shutil
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:

folder_path = '/content/drive/My Drive/Sabreena_data_model/'
x_test_output = 'x_test_1500_balanced.npy'
y_test_output = 'y_test_1500_balanced.npy'

os.makedirs("data",exist_ok=True)

shutil.copy(folder_path+x_test_output, "data/"+x_test_output)
shutil.copy(folder_path+y_test_output, "data/"+y_test_output)

'data/y_test_1500_balanced.npy'

In [ ]:
'''
path_lin_mod_opt = 'model/cp_lin_mod_opt_model/'
os.makedirs("model",exist_ok=True)
os.makedirs(path_lin_mod_opt,exist_ok=True)


for filename in os.listdir(folder_path+path_lin_mod_opt):
    source_file = os.path.join(folder_path+path_lin_mod_opt, filename)
    destination_file = os.path.join(path_lin_mod_opt, filename)
    if os.path.isfile(source_file):
        shutil.copy(source_file, destination_file)
'''

In [ ]:
from sklearn.ensemble import RandomForestRegressor
X=np.load("data/"+x_test_output)
y=np.load("data/"+y_test_output)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)
#x_test_500=x_test[:500]
#x_test_flat_500 = np.reshape(x_test_500/255, (47320000, 3)) #(141960000, 3)

X_train_300 = X_train[:300]
X_train_flat = np.reshape(X_train_300/255, (28392000, 3))
y_train_300=y_train[:300]
y_train_flat_300 = y_train_300.flatten()

X_test_flat = np.reshape(X_test/255, (28392000, 3))
y_test_300=y_test[:300]
y_test_flat_300 = y_test_300.flatten()


(1200, 338, 280, 3) (300, 338, 280, 3)


In [ ]:
# Build the model
def build_random_forest_regressor():
    model = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs = -1)
    return model

model_rf = build_random_forest_regressor()

# Train the model
model_rf.fit(X_train_flat, y_train_flat_300)



RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0)

In [ ]:
#full
X_train_flat_full = np.reshape(X_train/255, (113568000, 3))

# Build the model
def build_random_forest_regressor():
    model = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs = -1)
    return model

model_rf_full = build_random_forest_regressor()

# Train the model
model_rf_full.fit(X_train_flat_full, y_train.flatten())


RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0)

In [ ]:
#calculate mse

def calculate_mse(y_pred, y_true):

    mse = np.array([])

    y_pred_im = np.reshape(y_pred, (y_true.shape[0], 94640))
    y_true_im = np.reshape(y_true,  (y_true.shape[0], 94640))

    for i in range(y_pred_im.shape[0]):

        image_pred = y_pred_im[i]
        image_true = y_true_im[i]
        diff = np.subtract(image_true, image_pred)
        se = np.square(diff)
        mse_im = np.average(se)

        mse = np.append(mse, mse_im)

    mse_overall = np.average(mse)
    return mse_overall
# calculate r2

def calculate_r2(y_pred, y_true):

    r2_overall = r2_score(y_true, y_pred)

    return r2_overall
# calculate mae

def calculate_mae(y_pred, y_true):

    mae = np.array([])

    y_pred_im = np.reshape(y_pred, (y_true.shape[0], 94640))
    y_true_im = np.reshape(y_true,  (y_true.shape[0], 94640))

    for i in range(y_pred_im.shape[0]):

        image_pred = y_pred_im[i]
        image_true = y_true_im[i]

        mae_for_image = np.average(np.abs(np.subtract(image_true, image_pred)))

    mae = np.append(mae, mae_for_image)

    mae_overall = np.average(mae)

    return mae_overall

In [ ]:
y_test_flat = y_test.flatten()
y_pred_rf = model_rf_full.predict(X_test_flat)
mse = calculate_mse(y_pred_rf, y_test)
rmse = np.sqrt(mse)
mae = calculate_mae(y_pred_rf, y_test)
from sklearn.metrics import r2_score
r2 = calculate_r2(y_pred_rf, y_test_flat) # calculated_r2 requires flattened version of y_true

print("Testing MSE: ", mse)
print("Testing RMSE: ", rmse)
print("Testing MAE: ", mae)
print("Testing R2: ", r2)

Testing MSE:  102.29292497230792
Testing RMSE:  10.113996488644235
Testing MAE:  3.297115447840726
Testing R2:  0.9084469512805685


Total images: 1500
Train images: 1200
Test images: 300
Model used: Random Forest Regressor with parallel processing
No. of estimator in Random Forest: 10
Evaluation metric: MSE, MAE, R^2


Testing MSE:  102.29292497230792
Testing RMSE:  10.113996488644235
Testing MAE:  3.297115447840726
Testing R2:  0.9084469512805685

In [ ]:
y_test_flat = y_test.flatten()
y_pred_rf = model_rf_full.predict(X_train_flat_full)
mse = calculate_mse(y_pred_rf, y_train)
rmse = np.sqrt(mse)
mae = calculate_mae(y_pred_rf, y_train)
from sklearn.metrics import r2_score
r2 = calculate_r2(y_pred_rf, y_train.flatten()) # calculated_r2 requires flattened version of y_true

print("Training MSE: ", mse)
print("Training RMSE: ", rmse)
print("Training MAE: ", mae)
print("Training R2: ", r2)

Training MSE:  104.20482833674383
Training RMSE:  10.208076622789616
Training MAE:  3.937342411373155
Training R2:  0.9061307232858424
